# Model Spatial Regression

In [1]:
import pandas as pd
import numpy as np
import csv

#### Librerias específicas para la Autocorrelación espacial con Pysal

In [2]:
#For Spatial Correlation
# !pip install pysal
#http://pysal.readthedocs.io/en/latest/
import pysal as ps
from pysal.spreg import ols
from pysal.spreg import ml_error
from pysal.spreg import ml_lag

Recogemos el CSV procedente del Modelo Knn-vecinos con la variable Contaminante categórica codificada

In [3]:
csv_path_geom = '../data/csv/model_csv/df_model_geom_encoding.csv' #Recogemos el df Con geometrias para leer con geopandas
shp_path = '../data/shapes/Deaths/Deaths2015.shp'

In [4]:
#gdf = gpd.read_file(shp_path)
df = pd.read_csv(csv_path_geom, sep=';', encoding = 'utf-8', compression='gzip', index_col=False)

In [5]:
df.drop(['Unnamed: 0'], axis=1, inplace=True) #Borramos columnas innecesarias

In [6]:
pd.options.display.max_columns = None

In [7]:
df.columns

Index(['target', 'Sexo', 'AnioCumplidos', 'TamanioMuniResi',
       'total_anios_Expo_Id', 'Total_Kg_expo', 'Distance', 'geometry_death',
       'Latitud', 'Longitud', 'Habitantes', 'LatitudE', 'LongitudE',
       'Cont_Dióxido de carbono (CO2)', 'Cont_Partículas (PM10)'],
      dtype='object')

Observamos que todas las variables son numéricas pero habrá que escalar estas

In [8]:
df.dtypes

target                             int64
Sexo                               int64
AnioCumplidos                      int64
TamanioMuniResi                    int64
total_anios_Expo_Id                int64
Total_Kg_expo                    float64
Distance                           int64
geometry_death                    object
Latitud                          float64
Longitud                         float64
Habitantes                         int64
LatitudE                         float64
LongitudE                        float64
Cont_Dióxido de carbono (CO2)      int64
Cont_Partículas (PM10)             int64
dtype: object

Por conveniencia

In [9]:
df.rename(columns={'Cont_Dióxido de carbono (CO2)':'CO2_event'}, inplace=True)
df.rename(columns={'Cont_Partículas (PM10)':'PM10_event'}, inplace=True)

#### Pre-processing: Feature Scaling 

In [10]:
from sklearn.preprocessing import StandardScaler

#Standarization de columnas importantes
columnas = ['AnioCumplidos', 'TamanioMuniResi', 'total_anios_Expo_Id', 'Total_Kg_expo','Habitantes']
scaler = StandardScaler()
df[columnas] = scaler.fit_transform(df[columnas]) #Solo columnas de interés escaladas

In [11]:
df.head()

,target,Sexo,AnioCumplidos,TamanioMuniResi,total_anios_Expo_Id,Total_Kg_expo,Distance,geometry_death,Latitud,Longitud,Habitantes,LatitudE,LongitudE,CO2_event,PM10_event
0,1,1,-0.979792,-0.701283,1.237907,3.686373,4040,POINT (-5.348256 36.16117999999999),36.16118,-5.348256,-0.653751,36.181181,-5.385775,1,0
1,0,0,0.420983,-0.701283,1.237907,3.686373,4040,POINT (-5.348256 36.16117999999999),36.16118,-5.348256,-0.653751,36.181181,-5.385775,1,0
2,0,0,-0.463717,-0.701283,1.237907,3.686373,4040,POINT (-5.348256 36.16117999999999),36.16118,-5.348256,-0.653751,36.181181,-5.385775,1,0
3,0,0,-1.938217,-0.701283,1.237907,3.686373,4040,POINT (-5.348256 36.16117999999999),36.16118,-5.348256,-0.653751,36.181181,-5.385775,1,0
4,0,0,1.084508,-0.701283,1.237907,3.686373,4040,POINT (-5.348256 36.16117999999999),36.16118,-5.348256,-0.653751,36.181181,-5.385775,1,0


### Matrices de pesos espaciales (Spatial Weights) 

Debido a que el shape es muy pesado para realizar un análisis de regresión espacial primero vamos a realizar un muestreo aleatorio con numpy

In [12]:
rows = np.random.choice(df.index.values, 5000)

In [13]:
df_sam = df.loc[rows]

In [14]:
df_sam.shape

(5000, 15)

In [15]:
df_sam.head()

,target,Sexo,AnioCumplidos,TamanioMuniResi,total_anios_Expo_Id,Total_Kg_expo,Distance,geometry_death,Latitud,Longitud,Habitantes,LatitudE,LongitudE,CO2_event,PM10_event
64425,0,0,1.821759,0.051207,-0.435590,-0.522102,5883,POINT (2.099703999999997 41.35957999999999),41.35958,2.099704,-0.268968,41.388362,2.040820,0,1
26588,1,1,0.494708,0.803696,-0.435590,1.865319,417,POINT (-5.844759000000001 43.36026),43.36026,-5.844759,-0.335017,43.361915,-5.849389,1,0
11847,1,1,-1.938217,0.803696,0.680075,0.017392,1334,POINT (-0.8765379000000014 41.65629000000001),41.65629,-0.876538,0.565366,41.648823,-0.889085,1,0
31724,0,1,0.937058,-1.453772,0.122242,0.255137,1134,POINT (1.432378 38.90886),38.90886,1.432378,-0.685565,38.919060,1.431953,1,0
76055,0,1,0.199808,-1.453772,-0.993422,-0.522532,8395,POINT (-1.605718 42.06388),42.06388,-1.605718,-0.713491,42.124843,-1.665745,0,1


### k-nearest neighbor weights

Este método es una combinación de umbrales basados en la distancia junto con los pesos del kernel. En nuestro caso el ancho de banda se establece en un valor predeterminado y se fija a través de las observaciones

In [16]:
del df_sam['geometry_death']

En principio establecemos una matriz de pesos espaciales para k vecinos

In [17]:
wknn = ps.weights.KNN(df_sam, k = 30)

In [18]:
wknn.s0

150000.0

In [19]:
wknn.n

5000

Estandarizamos rows

In [20]:
wknn.transform = 'r'

## Modelos de Regresión

In [21]:
df_sam.columns

Index(['target', 'Sexo', 'AnioCumplidos', 'TamanioMuniResi',
       'total_anios_Expo_Id', 'Total_Kg_expo', 'Distance', 'Latitud',
       'Longitud', 'Habitantes', 'LatitudE', 'LongitudE', 'CO2_event',
       'PM10_event'],
      dtype='object')

Generamos los array

In [22]:
columns=['Sexo', 'AnioCumplidos','total_anios_Expo_Id', 'Total_Kg_expo','Habitantes','Distance']#se utiliza más adelante
np.random.seed(12345)
y = np.array(df_sam['target'])
y.shape = (len(y),1)
X= []
X.append(df_sam['Sexo'])
X.append(df_sam['AnioCumplidos'])
X.append(df_sam['total_anios_Expo_Id'])
X.append(df_sam['Total_Kg_expo'])
X.append(df_sam['Habitantes'])
X.append(df_sam['Distance'])

X = np.array(X).T

### Lineal Regression MCO (Nonspatial)

In [23]:
ls = ols.OLS(y, X, name_y = 'Causa fallecimiento CI10-34', name_x = columns\
             , name_ds = 'Dataframe Deaths sampled')
print(ls.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :Dataframe Deaths sampled
Weights matrix      :        None
Dependent Variable  :Causa fallecimiento CI10-34                Number of Observations:        5000
Mean dependent var  :      0.1596                Number of Variables   :           7
S.D. dependent var  :      0.3663                Degrees of Freedom    :        4993
R-squared           :      0.0705
Adjusted R-squared  :      0.0694
Sum squared residual:     623.378                F-statistic           :     63.0910
Sigma-square        :       0.125                Prob(F-statistic)     :   9.153e-76
S.E. of regression  :       0.353                Log likelihood        :   -1889.591
Sigma-square ML     :       0.125                Akaike info criterion :    3793.182
S.E of regression ML:      0.3531                Schwarz criterion     :    3838.802

--------------------------------------------------

#### Para ver la no independencia espacial, primero utilizamos la función I de Moran con los residuos desde un punto de vista global

Evaluamos la autocorrelación espacial en los residuos (ya que se supone que son independientes y no están correlacionados). 
Los valores de I pueden oscilar entre -1 (indicando dispersión perfecta) a 1 (correlación perfecta). Un valor de cero indica un patrón espacial aleatorio.

In [24]:
# Pasamos los weights matrix a la función pysal.Moran, a través de nuestro modelo residual (ls.u)
mi = ps.Moran(ls.u, wknn, two_tailed=False)
print('Observed I:', mi.I, '\nExpected I:', mi.EI, '\n   p-value:', mi.p_norm)

Observed I: 0.18082390404622783 
Expected I: -0.00020004000800160032 
   p-value: 0.0


### Spatial Regressión

Utilizamos la matriz de pesos espaciales k-nearest neighbor weights y la función de spatial Error model para cuantificar la no-independencia espacial

In [25]:
spat_err = ml_error.ML_Error(y, X, wknn, 
                             name_y='Causa fallecimiento CI10-34', name_x= ['Sexo','Años','Total años Expo','Kgs Cont.expo','Distance','Tamaño Municipio'], 
                             name_w='deaths wknn', name_ds='Dataframe Deaths sampled')
print(spat_err.summary)

C:\Users\julia\Anaconda3\lib\site-packages\scipy\optimize\_minimize.py:643: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  "defaulting to absolute tolerance.", RuntimeWarning)


REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL ERROR (METHOD = FULL)
-------------------------------------------------------------------
Data set            :Dataframe Deaths sampled
Weights matrix      : deaths wknn
Dependent Variable  :Causa fallecimiento CI10-34                Number of Observations:        5000
Mean dependent var  :      0.1596                Number of Variables   :           7
S.D. dependent var  :      0.3663                Degrees of Freedom    :        4993
Pseudo R-squared    :      0.0477
Sigma-square ML     :       0.098                Log likelihood        :   -1381.519
S.E of regression   :       0.312                Akaike info criterion :    2777.039
                                                 Schwarz criterion     :    2822.659

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
--------------------------------

#### Regresión lineal por Pysal con matriz de pesos espaciales

Realizamos otra Regresión mediante otro método del módulo spreg de Pysal

In [26]:
m1 = ps.spreg.OLS(y, X ,w=wknn, spat_diag=True, name_x=columns, name_y='target',\
                  name_w='deaths wknn',name_ds='Dataframe Deaths sampled')
print(m1.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :Dataframe Deaths sampled
Weights matrix      : deaths wknn
Dependent Variable  :      target                Number of Observations:        5000
Mean dependent var  :      0.1596                Number of Variables   :           7
S.D. dependent var  :      0.3663                Degrees of Freedom    :        4993
R-squared           :      0.0705
Adjusted R-squared  :      0.0694
Sum squared residual:     623.378                F-statistic           :     63.0910
Sigma-square        :       0.125                Prob(F-statistic)     :   9.153e-76
S.E. of regression  :       0.353                Log likelihood        :   -1889.591
Sigma-square ML     :       0.125                Akaike info criterion :    3793.182
S.E of regression ML:      0.3531                Schwarz criterion     :    3838.802

-----------------------------------------------------------------

#### Spatially lagged exogenous regressors

In [27]:
m3 = ps.spreg.GM_Lag(y, X, \
                  w=wknn, spat_diag=True, \
                  name_x=columns, name_y = 'Causa fallecimiento CI10-34',\
                     name_w='deaths wknn',name_ds='Dataframe Deaths sampled')
print(m3.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :Dataframe Deaths sampled
Weights matrix      : deaths wknn
Dependent Variable  :Causa fallecimiento CI10-34                Number of Observations:        5000
Mean dependent var  :      0.1596                Number of Variables   :           8
S.D. dependent var  :      0.3663                Degrees of Freedom    :        4992
Pseudo R-squared    :      0.2375
Spatial Pseudo R-squared:  0.0694

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT       0.0348508       0.0159059       2.1910699       0.0284467
                Sexo       0.1081678       0.0104058      10.3949426       0.0000000
       AnioCumplidos      -0.051